In [26]:
#importing necessary libraries
import pandas as pd #for data manipulation and analysis
import numpy as np #for mathematical functions
pd.set_option('display.max_columns',None) # to display all the columns
import seaborn as sns #for data visualisation
import matplotlib.pyplot as plt
import random
import ast

In [27]:
# reading the dataset
df = pd.read_csv(r'D:\BOOKS\UNIPI\Sem 2\Data Mining 2\Project\imdb.csv')
df

,originalTitle,rating,startYear,endYear,runtimeMinutes,awardWins,numVotes,worstRating,bestRating,totalImages,totalVideos,totalCredits,criticReviewsTotal,titleType,awardNominationsExcludeWins,canHaveEpisodes,isRatable,isAdult,numRegions,userReviewsTotal,ratingCount,countryOfOrigin,genres,castNumber,companiesNumber,averageRating,regions,externalLinks,writerCredits,directorsCredits,soundMixes,quotesTotal
0,Carmencita,"(5, 6]",1894,\N,1,0,2089,1,10,2,0,4,3,short,0,False,True,0,7,16,2092,['US'],"Documentary,Short",1,3,5.7,"['\\N', 'DE', 'US', 'HU', 'GR', 'RU', 'UA', 'JP']",9,0,1,['silent'],0
1,Un bon bock,"(5, 6]",1892,\N,12,0,183,1,10,2,0,2,0,short,0,False,True,0,6,1,183,['FR'],"Animation,Short",0,0,5.4,"['\\N', 'HU', 'DE', 'FR', 'RO', 'RU', 'JP']",3,0,1,['silent'],0
2,Chinese Opium Den,"(4, 5]",1894,\N,1,0,195,1,10,1,0,1,0,short,0,False,True,0,5,0,195,['US'],Short,0,1,5.0,"['\\N', 'US', 'DE', 'PT', 'HU', 'RU']",4,0,1,['silent'],0
3,Edison Kinetoscopic Record of a Sneeze,"(5, 6]",1894,\N,1,1,2237,1,10,3,0,4,2,short,0,False,True,0,6,20,2238,['US'],"Documentary,Short",1,6,5.4,"['\\N', 'UA', 'JP', 'HU', 'US', 'DE', 'RU']",6,0,1,['silent'],0
4,L'arrivée d'un train à La Ciotat,"(7, 8]",1896,\N,1,0,13115,1,10,12,0,11,16,short,0,False,True,0,21,66,13115,['FR'],"Documentary,Short",6,5,7.4,"['\\N', 'HU', 'DE', 'TR', 'FI', 'FR', 'IT', 'E...",21,0,2,['silent'],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149526,Nuestra película,"(6, 7]",1993,\N,96,0,11,1,10,0,0,11,0,movie,0,False,True,0,1,0,11,['CO'],Documentary,0,2,7.0,"['\\N', 'CO']",1,1,1,[],0
149527,Eco,"(6, 7]",2019,\N,14,0,15,1,10,4,1,52,1,short,3,False,True,0,1,0,15,['ES'],"Drama,Short",6,3,6.4,"['\\N', 'ES']",5,1,1,[],0
149528,Women Take Center Stage,"(5, 6]",2019,\N,\N,0,12,1,10,2,3,32,0,tvEpisode,0,False,True,0,0,0,12,NaN,Reality-TV,1,0,5.6,['\\N'],0,0,0,[],0
149529,Horrid Henry and the Christening Crisis,"(7, 8]",2011,\N,10,0,9,1,10,1,0,15,0,tvEpisode,0,False,True,0,0,0,10,NaN,"Adventure,Animation,Comedy",7,0,7.1,['\\N'],0,3,1,[],0


In [28]:
# making a dataframe with no runtime error
df_copy = df.copy()
df_copy.drop(df[df['runtimeMinutes'] == '\\N'].index, inplace = True)
df_copy['runtimeMinutes'] = pd.to_numeric(df_copy['runtimeMinutes'])
df_copy.shape

(109336, 32)

In [29]:
# Compute IQR for each titleType feature for runtimeMinutes
iqr_values = {}
k = 1.5
for type in df_copy['titleType'].unique():
    Q1 = int(df_copy[df_copy['titleType']==type]['runtimeMinutes'].quantile(0.25))  # First quartile (25%)
    Q3 = int(df_copy[df_copy['titleType']==type]['runtimeMinutes'].quantile(0.75))  # Third quartile (75%)
    IQR = Q3 - Q1  # Interquartile Range
    iqr_values[type] = [type, Q1, Q3, IQR, Q1 - k*IQR, Q3 + k*IQR]

# Convert dictionary to DataFrame for better visualization
iqr_df_title = pd.DataFrame.from_dict(iqr_values, orient = 'index', columns=['titleType', 'Q1','Q3','IQR', 'Min Outlier Value', 'Max Outlier Value']).reset_index(drop=True)

# Print the IQR values
iqr_df_title

,titleType,Q1,Q3,IQR,Min Outlier Value,Max Outlier Value
0,short,7,19,12,-11.0,37.0
1,movie,80,102,22,47.0,135.0
2,tvSeries,28,60,32,-20.0,108.0
3,tvEpisode,24,48,24,-12.0,84.0
4,tvMovie,59,96,37,3.5,151.5
5,tvMiniSeries,45,174,129,-148.5,367.5
6,tvShort,6,18,12,-12.0,36.0
7,video,30,91,61,-61.5,182.5
8,tvSpecial,59,120,61,-32.5,211.5
9,videoGame,30,97,67,-70.5,197.5


In [30]:
# imputing random integer values between the 25th percentile and 75th percentile
for i in range(len(df)):
    if df['runtimeMinutes'][i] == '\\N': 
        type = df['titleType'][i]        
        df.loc[i, 'runtimeMinutes'] = random.randrange(iqr_df_title.loc[iqr_df_title['titleType'] == type, 'Q1'].values[0], iqr_df_title.loc[iqr_df_title['titleType'] == type, 'Q3'].values[0])
        

In [31]:
#changing the runtime to numeric data type
df['runtimeMinutes'] = pd.to_numeric(df['runtimeMinutes'])

In [32]:
#dropping unwanted columns
df.drop(columns = ['originalTitle', 'isAdult', 'endYear','numVotes','worstRating', 'bestRating', 'isRatable','rating','soundMixes'], inplace = True) # you can drop totalCredits as well

In [33]:
#merging columns
df['totalNominations'] = df['awardNominationsExcludeWins'] + df['awardWins']
df['totalMedia'] = df['totalImages']+ df['totalVideos'] + df['quotesTotal'] + df['externalLinks']
df['totalReviews'] = df['criticReviewsTotal'] + df['userReviewsTotal']

In [34]:
# dropping the merged columns
df.drop(columns = ['awardNominationsExcludeWins', 'awardWins', 'totalImages', 'totalVideos', 'quotesTotal','externalLinks', 'criticReviewsTotal', 'userReviewsTotal'], inplace = True)
df.shape

(149531, 18)

In [35]:
def clean_region_string(region_str):
    try:
        # Convert string to actual list
        region_list = ast.literal_eval(region_str)
        
        # Remove '\\N' and return cleaned list
        return [r for r in region_list if r != '\\N']
    except Exception as e:
        # If conversion fails, return an empty list or NaN
        return []

# Apply to the column
df['regions'] = df['regions'].apply(clean_region_string)
df['countryOfOrigin'] = df['countryOfOrigin'].apply(clean_region_string)

In [36]:
# Check if origin is subset of regions
def is_subset(row):
    return set(row['countryOfOrigin']).issubset(set(row['regions']))

# Apply and count
df['is_subset'] = df.apply(is_subset, axis=1)
subset_count = df['is_subset'].sum()

print(f"Number of rows where countryOfOrigin is a subset of regions: {subset_count}")

Number of rows where countryOfOrigin is a subset of regions: 141728


In [37]:
# Create a new column with the union of the two
df['all_countries'] = df.apply(lambda row: list(set(row['regions']) | set(row['countryOfOrigin'])), axis=1)

In [38]:
%%time
import pycountry_convert as pc

# Define continent buckets
continent_columns = ['Asia', 'Africa', 'Europe', 'North America', 'South America', 'Australia', 'Continent Unknown']

def get_continent(country_code):
    try:
        country_alpha2 = country_code.upper()
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2) # Convert to continent code
        continent_map = {
            'AS': 'Asia',
            'AF': 'Africa',
            'EU': 'Europe',
            'NA': 'North America',
            'SA': 'South America',
            'OC': 'Australia'
        }
        return continent_map.get(continent_code, 'Continent Unknown')
    except:
        return 'Continent Unknown'

def frequency_encode_continents(row):
    freq_dict = {continent: 0 for continent in continent_columns}
    for code in row['all_countries']:
        continent = get_continent(code)
        freq_dict[continent] += 1
    return pd.Series(freq_dict)

# Apply to your dataframe
continent_freq_df = df.apply(frequency_encode_continents, axis=1)

# Merge the frequency columns back to your original dataframe
df = pd.concat([df, continent_freq_df], axis=1)


CPU times: total: 35.2 s
Wall time: 47.3 s


In [39]:
from collections import Counter

# Step 1: Convert genres to list
df['genres_list'] = df['genres'].fillna('').apply(lambda x: x.split(',')[:3])

# Step 2: Count frequencies
all_genres = df['genres_list'].sum()
genre_freq = Counter(all_genres)
genre_freq

Counter({'Drama': 51745,
         'Comedy': 47408,
         'Action': 18931,
         'Animation': 17855,
         'Adventure': 17763,
         'Short': 16787,
         'Documentary': 16581,
         'Crime': 16437,
         'Family': 13233,
         'Romance': 11665,
         'Mystery': 6868,
         'Fantasy': 6144,
         'Horror': 5785,
         'Reality-TV': 5726,
         'Thriller': 5700,
         'Music': 4788,
         'History': 4088,
         'Sci-Fi': 3839,
         'Talk-Show': 3370,
         'Game-Show': 3019,
         'Western': 2808,
         'Biography': 2791,
         'Sport': 2702,
         '\\N': 2691,
         'Adult': 2642,
         'War': 2063,
         'Musical': 1870,
         'News': 1671,
         'Film-Noir': 155})

In [40]:
#setting the frequency of '\\N' to 0
genre_freq['\\N'] = 0

In [41]:
# Step 3: Sort each row’s genres by overall frequency
def sort_genres_by_frequency(genres):
    return sorted(genres, key=lambda g: -genre_freq.get(g, 0))

df['sorted_genres'] = df['genres_list'].apply(sort_genres_by_frequency)

In [42]:
# Step 4: Assign frequencies
df['genre1'] = df['sorted_genres'].apply(lambda x: genre_freq.get(x[0], 0) if len(x) > 0 else 0)
df['genre2'] = df['sorted_genres'].apply(lambda x: genre_freq.get(x[1], 0) if len(x) > 1 else 0)
df['genre3'] = df['sorted_genres'].apply(lambda x: genre_freq.get(x[2], 0) if len(x) > 2 else 0)

In [43]:
# Convert canHaveEpisodes to boolean data type
df['canHaveEpisodes'] = df['canHaveEpisodes'].astype(bool).astype(int)

In [44]:
# One-hot encode the 'titleType' column
titleType_dummies = pd.get_dummies(df['titleType']).astype(bool).astype(int)

# Concatenate with the original dataframe
df = pd.concat([df, titleType_dummies], axis=1)

In [45]:
# Drop the original columns
df.drop(columns = ['titleType', 'countryOfOrigin', 'genres', 'regions', 'all_countries', 'is_subset','genres_list', 'sorted_genres'], inplace = True)
df.shape

(149531, 34)

In [46]:
df

,startYear,runtimeMinutes,totalCredits,canHaveEpisodes,numRegions,ratingCount,castNumber,companiesNumber,averageRating,writerCredits,directorsCredits,totalNominations,totalMedia,totalReviews,Asia,Africa,Europe,North America,South America,Australia,Continent Unknown,genre1,genre2,genre3,movie,short,tvEpisode,tvMiniSeries,tvMovie,tvSeries,tvShort,tvSpecial,video,videoGame
0,1894,1,4,0,7,2092,1,3,5.7,0,1,0,11,19,1,0,5,1,0,0,0,16787,16581,0,0,1,0,0,0,0,0,0,0,0
1,1892,12,2,0,6,183,0,0,5.4,0,1,0,5,1,1,0,5,0,0,0,0,17855,16787,0,0,1,0,0,0,0,0,0,0,0
2,1894,1,1,0,5,195,0,1,5.0,0,1,0,5,0,0,0,4,1,0,0,0,16787,0,0,0,1,0,0,0,0,0,0,0,0
3,1894,1,4,0,6,2238,1,6,5.4,0,1,1,9,22,1,0,4,1,0,0,0,16787,16581,0,0,1,0,0,0,0,0,0,0,0
4,1896,1,11,0,21,13115,6,5,7.4,0,2,0,33,82,2,0,14,2,1,0,2,16787,16581,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149526,1993,96,11,0,1,11,0,2,7.0,1,1,0,1,0,0,0,0,0,1,0,0,16581,0,0,1,0,0,0,0,0,0,0,0,0
149527,2019,14,52,0,1,15,6,3,6.4,1,1,3,10,1,0,0,1,0,0,0,0,51745,16787,0,0,1,0,0,0,0,0,0,0,0
149528,2019,29,32,0,0,12,1,0,5.6,0,0,0,5,0,0,0,0,0,0,0,0,5726,0,0,0,0,1,0,0,0,0,0,0,0
149529,2011,10,15,0,0,10,7,0,7.1,3,1,0,1,0,0,0,0,0,0,0,0,47408,17855,17763,0,0,1,0,0,0,0,0,0,0


In [47]:
df.to_csv('imdb_2.csv', index=False) # Save the cleaned DataFrame to a new CSV file